In [58]:
import urllib
import json
import pandas as pd
import vertica_python
import re
from datetime import datetime, timedelta

In [4]:
x1 = datetime(2020,1,1)
x2 = datetime(2020,1,18)
d = []
while x1<=x2:
    d.append(x1.strftime('%Y-%m-%d'))
    x1 += timedelta(days=1)
    
print(d)

['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18']


In [76]:
#Забираем файл и преобразовываем его в json.
#url = 'https://www.cbr-xml-daily.ru//archive//2020//11//11//daily_json.js'
url = 'https://www.cbr-xml-daily.ru/daily_json.js'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

In [60]:
valute = ('USD', 'EUR', 'CNY', 'INR', 'JPY')
delimiter = '\x1F'
line_terminator = '\x1E'

In [90]:
table_header = "stg_valute.valute(ID, NumCode, CharCode, Nominal, Name, Value, Date_)"
rej_table = 'stg_valute.rej_table_valute'

In [62]:
connection_info = {'host': 'localhost',
             'port': 5433,
             'user': 'dbadmin',
             'password': '',
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': False,
             # autocommit is off by default
             'autocommit': True,
             # using server-side prepared statements is disabled by default
             'use_prepared_statements': False,
             # connection timeout is not enabled by default
             # 5 seconds timeout for a socket operation (Establishing a TCP connection or read/write operation)
             'connection_timeout': 5}


In [6]:
#This code only for experiment ma'faka
#with open('valute.csv', 'w', newline='', encoding='utf8') as csvfile:
    #data_writer = csv.writer(csvfile, delimiter = '\x1F', lineterminator = '\x1E')
    #for i in valute:
        #for j in atribute:
            #a = str(data['Valute'][i][j])
            #data_writer.writerow([a]) 
            #print(type(a), a)

In [48]:
with vertica_python.connect(**connection_info) as connection:
        cur = connection.cursor()
        cur.execute("START TRANSACTION;")
        cur.execute("SET SESSION AUTOCOMMIT TO OFF;")
        a = cur.execute('select count(*) from valute.valute;').fetchall()
        print(type(a[0][0]), '|' , a[0][0])

<class 'int'> | 34


In [55]:
data

{'Date': '2010-06-01T11:30:00+04:00',
 'PreviousDate': '2010-05-29T11:30:00+04:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2010/05/29/daily_json.js',
 'Timestamp': '2010-06-01T11:30:00+04:00',
 'Valute': {'AUD': {'ID': 'R01010',
   'NumCode': '036',
   'CharCode': 'AUD',
   'Nominal': 1,
   'Name': 'Австралийский доллар',
   'Value': 26.0583,
   'Previous': 25.9487},
  'AZN': {'ID': 'R01020A',
   'NumCode': '944',
   'CharCode': 'AZN',
   'Nominal': 1,
   'Name': 'Азербайджанский манат',
   'Value': 38.2529,
   'Previous': 37.944},
  'GBP': {'ID': 'R01035',
   'NumCode': '826',
   'CharCode': 'GBP',
   'Nominal': 1,
   'Name': 'Фунт стерлингов Соединенного королевства',
   'Value': 44.5392,
   'Previous': 44.365},
  'AMD': {'ID': 'R01060',
   'NumCode': '051',
   'CharCode': 'AMD',
   'Nominal': 1000,
   'Name': 'Армянских драмов',
   'Value': 81.0547,
   'Previous': 80.4633},
  'BYR': {'ID': 'R01090',
   'NumCode': '974',
   'CharCode': 'BYR',
   'Nominal': 1000,
   'Name': '

In [81]:
df = pd.DataFrame(data['Valute'])
df = df.T
df = df.loc[df['CharCode'].isin(valute)]

In [13]:
df2 = pd.DataFrame(data2['Valute'])
current_date = data2['Date']
df2 = df2.T

In [ ]:
df2 = pd.DataFrame(data2['Valute'])
current_date = data2['Date']
df2 = df2.T

In [78]:
df2.loc[df2['CharCode'].isin(valute)]

,ID,NumCode,CharCode,Nominal,Name,Value,Date
USD,R01235,840,USD,1,Доллар США,76.0713,2020-09-10
EUR,R01239,978,EUR,1,Евро,89.5663,2020-09-10
INR,R01270,356,INR,10,Индийских рупий,10.3429,2020-09-10
CNY,R01375,156,CNY,1,Китайский юань,11.1118,2020-09-10
JPY,R01820,392,JPY,100,Японских иен,71.7688,2020-09-10


In [73]:
re.match(r'\d{4}-\d{2}-\d{2}',current_date).group(0)

'2020-09-10'

In [82]:
df.drop(['Previous'], axis = 1, inplace = True)
df['Nominal'] = df['Nominal'].astype(int)
df['Value'] = df['Value'].astype(float)
df['Date_'] = re.match(r'\d{4}-\d{2}-\d{2}',current_date).group(0)

In [83]:
df


,ID,NumCode,CharCode,Nominal,Name,Value,Date_
USD,R01235,840,USD,1,Доллар США,76.3978,2020-09-10
EUR,R01239,978,EUR,1,Евро,90.3557,2020-09-10
INR,R01270,356,INR,10,Индийских рупий,10.3014,2020-09-10
CNY,R01375,156,CNY,1,Китайский юань,11.5689,2020-09-10
JPY,R01820,392,JPY,100,Японских иен,72.7217,2020-09-10


In [28]:
z = df.append(df2)

In [33]:
d = pd.DataFrame()


In [34]:
d = z.append(df)

In [35]:
d

,ID,NumCode,CharCode,Nominal,Name,Value,Date
AUD,R01010,036,AUD,1,Австралийский доллар,56.0725,2020-10-06
AZN,R01020A,944,AZN,1,Азербайджанский манат,45.9848,2020-10-06
GBP,R01035,826,GBP,1,Фунт стерлингов Соединенного королевства,101.1603,2020-10-06
AMD,R01060,051,AMD,100,Армянских драмов,15.9445,2020-10-06
BYN,R01090B,933,BYN,1,Белорусский рубль,29.8119,2020-10-06
...,...,...,...,...,...,...,...
SEK,R01770,752,SEK,10,Шведских крон,87.7371,2020-10-06
CHF,R01775,756,CHF,1,Швейцарский франк,85.1254,2020-10-06
ZAR,R01810,710,ZAR,10,Южноафриканских рэндов,47.5877,2020-10-06
KRW,R01815,410,KRW,1000,Вон Республики Корея,67.2999,2020-10-06


In [84]:
df.to_csv('valute.csv', index=False, sep = delimiter, line_terminator = line_terminator,
            header=True,
            encoding='utf-8',
            na_rep=' ')

In [93]:
with vertica_python.connect(**connection_info) as connection:
        copy_req = """COPY {table_header}
                FROM STDIN PARSER
                fcsvparser(
                    type='traditional',
                    delimiter='{delimiter}',
                    header='true',
                    record_terminator='{line_terminator}',
                    reject_on_materialized_type_error = 'true'
                ) REJECTED DATA AS TABLE {rej_table};
            """.format(
            table_header=table_header,
            delimiter=delimiter,
            line_terminator=line_terminator,
            rej_table=rej_table
        )
        cur = connection.cursor()
        cur.execute("START TRANSACTION;")
        cur.execute("SET SESSION AUTOCOMMIT TO OFF;")
        with open('C:\\Users\\adyan.beltikov\\Desktop\\programs\\Glob_currency\\valute.csv', "rb") as fs:
            try:
                cur.copy(copy_req, fs, buffer_size=65536)
                cur.execute('select get_num_rejected_rows();')
                after = cur.fetchone()
                print(after)
                cur.execute("COMMIT;")
            except Exception as ex:
                print(ex)

[5576]
